In [ ]:
%pip install git+https://github.com/m-bain/whisperx.git
%pip install whisperx-karaoke

In [1]:
dir_path = "./raw"
device = "cuda"
language = "ja"
offset = 0

In [ ]:
import os
import whisperx
from whisperx_karaoke.resolver import parse_file
from whisperx_karaoke.ass import segments_to_ass_text
from whisperx_karaoke.cli import get_audio_and_song_path, use_lrc_start_and_end

audio_path, lrc_path = get_audio_and_song_path(dir_path)
audio = whisperx.load_audio(audio_path)
cleared = parse_file(lrc_path, language, offset)
# print(cleared)
for i in range(len(cleared["segments"])):
    index = len(cleared["segments"]) - 1 - i
    if cleared["segments"][index]["text"].strip() == "":
        # print(i)
        cleared["segments"].pop(index)
model_a, metadata = whisperx.load_align_model(
    language_code=cleared["language"], device=device
)
result = whisperx.align(
    cleared["segments"],
    model_a,
    metadata,
    audio,
    device,
    return_char_alignments=False,
)

use_lrc_start_and_end(result["segments"], cleared["segments"])

ass_output = segments_to_ass_text(result["segments"])
ass_filename = os.path.join(dir_path, f"{os.path.basename(audio_path)}.ass")
with open(ass_filename, "w", encoding="utf-8") as f:
    f.write(ass_output)
    print("ASS file saved to: ", ass_filename)